# Prueba de ciclo circadiano 2

In [1]:
import pandas as pd 
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
def extract_circadian_cycle(data, sample_rate=24, z=1):
    
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/sample_rate)
    
    # Mantener solo los armónicos en el rango [-z, z]
    fft_filtered = np.zeros_like(fft_result, dtype=complex)
    for k in range(-z, z + 1):
        idx = np.argmin(np.abs(freq - k))  # Encuentra el índice más cercano a la frecuencia k
        fft_filtered[idx] = fft_result[idx]
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))

# Función para procesar una columna de actividad y calcular niveles de estrés
def process_activity(df, activity_column,nombre_csv):
    # Crear ventanas móviles de 36 horas con paso de 1 hora
    window_size = 36
    step_size = 1

    series_results = []
    for start_idx in range(0, len(df) - window_size + 1, step_size):
        window_data = df.iloc[start_idx:start_idx + window_size]
        if len(window_data) == window_size:
            # Subseries A y B de 24 horas con desfase de 12 horas
            subseries_A = window_data.iloc[:24]
            subseries_B = window_data.iloc[12:36]
            
            # Extraer ciclos circadianos
            cycle_A = extract_circadian_cycle(subseries_A[activity_column].values)
            cycle_B = extract_circadian_cycle(subseries_B[activity_column].values)
            # Sincronizar modelos considerando el desfase de 12 horas
            synced_A = {i: cycle_A[i] for i in range(12, 24)}  # Ciclo A desde la hora 12 a 23
            synced_B = {i+12: cycle_B[i] for i in range(0, 12)}  # Ciclo B desde la hora 0 a 12
            
            # print('Cycle a')
            # print(cycle_A)
            # print('Cycle b')
            # print(cycle_B)
            
            # Comparar ciclos sincronizados (clave a clave)
            differences = []
            for key in synced_A:
                
                if key in synced_B:  # Verificar que exista la clave en ambos ciclos
                    differences.append((synced_A[key] - synced_B[key]) ** 2)
                    #print(differences)
                else:
                    differences.append(np.abs(synced_A[key] - 0))
            
            # Calcular distancia euclidiana acumulada
            dist_euclidiana = np.sqrt(sum(differences))
            #print(dist_euclidiana)
            
            # Almacenar resultados
            series_results.append({
                'Fecha': window_data.index[24].date(),
                'Hora': window_data.index[24].hour,
                'distancia': dist_euclidiana
            })

    # Guardar resultados como DataFrame
    distance_df = pd.DataFrame(series_results)
    
    # Establecer umbrales de estrés usando desviación estándar
    mean_dist = distance_df['distancia'].mean()
    std_dist = distance_df['distancia'].std()
    threshold1 = mean_dist + std_dist
    threshold2 = mean_dist + 2 * std_dist
    threshold_t.append([nombre_csv,threshold1,threshold2])
        
    # Clasificación de niveles de estrés basada en los umbrales
    def get_stress_level(distance):
        if distance <= threshold1:
            return 'Normal'
        elif distance <= threshold2:
            return 'Leve'
        else:
            return 'Alto'

    # Aplicar la función de nivel de estrés a cada distancia
    distance_df['nivel_estres'] = distance_df['distancia'].apply(get_stress_level)
    
    return distance_df[['Fecha', 'Hora','distancia', 'nivel_estres']]
    
threshold_t = []            
# Configuración de datos de entrada
nombres_csv = {1, 2, 3, 4, 468, 479, 4003, 4151, 4160, 4173}

for nombre_csv in nombres_csv:
    print(nombre_csv)
    df = pd.read_csv(f'Datos_actividad/{nombre_csv}_act.csv')
    # fecha_limite = '2024-01-02'
    # df = data[data['Fecha'] <= fecha_limite].copy()
    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()
    
    # Procesar  columnas de actividad y calcular niveles de estrés
    stress_levels = process_activity(df, 'nivel_actividad',nombre_csv)
    # Fusionar niveles de estrés con el DataFrame original
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    stress_levels['Fecha'] = pd.to_datetime(stress_levels['Fecha'])
    df['Hora'] = df['Hora'].astype(int)
    stress_levels['Hora'] = stress_levels['Hora'].astype(int)
    
    # Fusionar todos los DataFrames en df_final
    df_final = df.merge(stress_levels, on=['Fecha', 'Hora'], how='left')
    df_final = df_final.iloc[24:-24]
    #Definir la fecha límite como Timestamp
    fecha_limite = pd.Timestamp('2024-06-30')

    # Dividir el DataFrame en dos partes
    df_entrenamiento = df_final[df_final['Fecha'] <= fecha_limite].copy()  # DataFrame antes de la fecha límite
    df_prueba = df_final[df_final['Fecha'] > fecha_limite].copy()  # DataFrame desde la fecha límite en adelante

    # Guardar los DataFrames en archivos CSV
    df_entrenamiento.to_csv(f'Datos_entrenamiento2/{nombre_csv}_e.csv', index=False) 
    df_prueba.to_csv(f'Datos_prueba2/{nombre_csv}_p.csv', index=False)

    
    


4160
1
2
3
4
4003
4173
468
4151
479


In [2]:
# # Contamos las ocurrencias de cada nivel de estrés en la nueva columna
# conteo_nivel_estres = df_final['nivel_estres'].value_counts()

# print(conteo_nivel_estres)

In [3]:
threshold_t

[[4160, 0.1654913921088168, 0.23390124573130353],
 [1, 0.16219042140329884, 0.22378973068163877],
 [2, 0.22519997161114347, 0.3073499690525403],
 [3, 0.23292124350670784, 0.3196846998304068],
 [4, 0.24643035083244613, 0.35418289947652337],
 [4003, 0.15438802372192764, 0.2118139210921349],
 [4173, 0.15994222333220132, 0.21935511514786213],
 [468, 0.23956998815505537, 0.32874140151918524],
 [4151, 0.2591016453499819, 0.3583282368901059],
 [479, 0.19259006383804755, 0.26351707911210387]]

In [4]:
# Guardar threshold1_t como un archivo CSV
threshold_v_df = pd.DataFrame(threshold_t, columns=['nombre_csv','threshold1','threshold2'])
threshold_v_df.to_csv('umbrales/threshold_values.csv', index=False)